In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
links = pd.read_csv('poster_links.csv')

In [4]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [5]:
movies.shape

(4803, 20)

In [6]:
movies = movies.merge(links, on = 'id')

In [7]:
movies.shape

(4797, 24)

In [8]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'Unnamed: 0', 'poster_link_w185', 'poster_link_w342',
       'poster_link_w154'],
      dtype='object')

In [9]:
popularMovies = movies[['id','title','popularity','vote_average','vote_count','poster_link_w185','poster_link_w342','poster_link_w154']]

In [10]:
popularMovies.isnull().sum()

id                  0
title               0
popularity          0
vote_average        0
vote_count          0
poster_link_w185    0
poster_link_w342    0
poster_link_w154    0
dtype: int64

In [11]:
popularMovies.reset_index(drop=True, inplace = True)

In [12]:


popularMovies['vote_count'] = popularMovies['vote_count'].astype('int')
popularMovies['vote_average'] = popularMovies['vote_average'].astype('int')

vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')

C = vote_averages.mean()
m = vote_counts.quantile(0.50)
popularMovies.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_4592\1869101999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popularMovies['vote_count'] = popularMovies['vote_count'].astype('int')
C:\Users\Admin\AppData\Local\Temp\ipykernel_4592\1869101999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popularMovies['vote_average'] = popularMovies['vote_average'].astype('int')


(4797, 8)

In [13]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
popularMovies['wr'] = popularMovies.apply(weighted_rating, axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_4592\3131478990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popularMovies['wr'] = popularMovies.apply(weighted_rating, axis=1)


In [15]:
popularMovies.head(5)

,id,title,popularity,vote_average,vote_count,poster_link_w185,poster_link_w342,poster_link_w154,wr
0,19995,Avatar,150.437577,7,11800,https://image.tmdb.org/t/p/w185//jRXYjXNq0Cs2T...,https://image.tmdb.org/t/p/w342//jRXYjXNq0Cs2T...,https://image.tmdb.org/t/p/w154//jRXYjXNq0Cs2T...,6.973534
1,285,Pirates of the Caribbean: At World's End,139.082615,6,4500,https://image.tmdb.org/t/p/w185//jGWpG4YhpQwVm...,https://image.tmdb.org/t/p/w342//jGWpG4YhpQwVm...,https://image.tmdb.org/t/p/w154//jGWpG4YhpQwVm...,5.982780
2,206647,Spectre,107.376788,6,4466,https://image.tmdb.org/t/p/w185//672kUEMtTHcaV...,https://image.tmdb.org/t/p/w342//672kUEMtTHcaV...,https://image.tmdb.org/t/p/w154//672kUEMtTHcaV...,5.982656
3,49026,The Dark Knight Rises,112.312950,7,9106,https://image.tmdb.org/t/p/w185//85cWkCVftiVs0...,https://image.tmdb.org/t/p/w342//85cWkCVftiVs0...,https://image.tmdb.org/t/p/w154//85cWkCVftiVs0...,6.965903
4,49529,John Carter,43.926995,6,2124,https://image.tmdb.org/t/p/w185//lCxz1Yus07QCQ...,https://image.tmdb.org/t/p/w342//lCxz1Yus07QCQ...,https://image.tmdb.org/t/p/w154//lCxz1Yus07QCQ...,5.965451


In [16]:
popularMovies = popularMovies.sort_values('wr', ascending=False)

In [17]:
popularMovies.head(5)

,id,title,popularity,vote_average,vote_count,poster_link_w185,poster_link_w342,poster_link_w154,wr
96,27205,Inception,167.583710,8,13752,https://image.tmdb.org/t/p/w185//edv5CZvWj09up...,https://image.tmdb.org/t/p/w342//edv5CZvWj09up...,https://image.tmdb.org/t/p/w154//edv5CZvWj09up...,7.960285
65,155,The Dark Knight,187.322927,8,12002,https://image.tmdb.org/t/p/w185//qJ2tW6WMUDux9...,https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux9...,https://image.tmdb.org/t/p/w154//qJ2tW6WMUDux9...,7.954607
95,157336,Interstellar,724.247784,8,10867,https://image.tmdb.org/t/p/w185//gEU2QniE6E77N...,https://image.tmdb.org/t/p/w342//gEU2QniE6E77N...,https://image.tmdb.org/t/p/w154//gEU2QniE6E77N...,7.949967
662,550,Fight Club,146.757391,8,9413,https://image.tmdb.org/t/p/w185//pB8BM7pdSp6B6...,https://image.tmdb.org/t/p/w342//pB8BM7pdSp6B6...,https://image.tmdb.org/t/p/w154//pB8BM7pdSp6B6...,7.942428
262,120,The Lord of the Rings: The Fellowship of the Ring,138.049577,8,8705,https://image.tmdb.org/t/p/w185//6oom5QYQ2yQTM...,https://image.tmdb.org/t/p/w342//6oom5QYQ2yQTM...,https://image.tmdb.org/t/p/w154//6oom5QYQ2yQTM...,7.937870


In [19]:
import pickle
pickle.dump(popularMovies, open("TopRatedMovies.pkl",'wb'))

In [ ]:
# popularMovies 